# Achievement 2.2: Sourcing Web Data with an API

This notebook collects and combines Citi Bike trip data and NOAA weather data for New York City in 2022. The goal is to prepare a clean dataset for use in an interactive dashboard that helps identify distribution issues in the Citi Bike system by analyzing ridership behavior in relation to date, time, station usage, and weather conditions.

## Table of Contents
1. [Imports and Setup](#1.-Imports-and-Setup)
2. [Data Load and Consolidation](#2.-Data-Load-and-Consolidation)
3. [NOAA API: Weather Data Extraction](#3.-NOAA-API:-Weather-Data-Extraction)
4. [Retrying Failed Weather Requests](#4.-Retrying-Failed-Weather-Requests)
5. [Combining Weather Files](#5.-Combining-Weather-Files)
6. [Merging Citi Bike and Weather Data](#6.-Merging-Citi-Bike-and-Weather-Data)
7. [Observations and Export](#7.-Observations-and-Export)

## 1. Imports and Setup

In [ ]:
import os
import glob
import pandas as pd
import requests
from time import sleep
from dotenv import load_dotenv

## 2. Data Load and Consolidation

In [ ]:
data_folder = r'C:\Users\rewha\Desktop\0.CareerFoundry\3.Data Vis w-Python_ACH 02-Dashboards w-Pythons\2022-citibike-tripdata\2022-citibike-tripdata'
csv_files = glob.glob(os.path.join(data_folder, '*.csv'))

dfs = []
for file in csv_files:
    try:
        df = pd.read_csv(file, engine='c', low_memory=False)
        df.columns = [col.lower().strip() for col in df.columns]
        dfs.append(df)
        print(f"✅ Loaded: {os.path.basename(file)} → Shape: {df.shape}")
    except Exception as e:
        print(f"❌ Skipped: {os.path.basename(file)} → Error: {e}")

if dfs:
    combined_df = pd.concat(dfs, ignore_index=True)
    print("✅ Combined Shape:", combined_df.shape)
    combined_df.to_csv("combined_citibike_2022.csv", index=False)
else:
    print("❌ No dataframes to concatenate.")

## 3. NOAA API: Weather Data Extraction

In [ ]:
token = "your_token_here"  # Replace with your actual token
headers = {'token': token}
station_id = "GHCND:USW00014732"
base_url = "https://www.ncei.noaa.gov/cdo-web/api/v2/data"

months = [
    ("2022-01-01", "2022-01-31"), ("2022-02-01", "2022-02-28"),
    ("2022-03-01", "2022-03-31"), ("2022-04-01", "2022-04-30"),
    ("2022-05-01", "2022-05-31"), ("2022-06-01", "2022-06-30"),
    ("2022-07-01", "2022-07-31"), ("2022-08-01", "2022-08-31"),
    ("2022-09-01", "2022-09-30"), ("2022-10-01", "2022-10-31"),
    ("2022-11-01", "2022-11-30"), ("2022-12-01", "2022-12-31")
]

all_data = []
for start, end in months:
    print(f"Fetching {start} to {end}...")
    params = {
        "datasetid": "GHCND",
        "stationid": station_id,
        "startdate": start,
        "enddate": end,
        "limit": 1000,
        "units": "standard",
        "datatypeid": ["TMAX", "TMIN", "PRCP"],
    }
    response = requests.get(base_url, headers=headers, params=params)
    if response.status_code == 200:
        all_data.extend(response.json().get("results", []))
        print(f"✅ Success: {len(response.json().get('results', []))} records")
    else:
        print(f"❌ Failed: {response.status_code} - {response.text}")
    sleep(1)

weather_df = pd.DataFrame(all_data)
weather_df['date'] = pd.to_datetime(weather_df['date'])
weather_wide = weather_df.pivot_table(index='date', columns='datatype', values='value').reset_index()
weather_wide.to_csv("laguardia_weather_2022.csv", index=False)

## 4. Retrying Failed Weather Requests

In [ ]:
retry_months = [("2022-02-01", "2022-02-28"), ("2022-12-01", "2022-12-31")]
extra_data = []
for start, end in retry_months:
    print(f"Retrying {start} to {end}...")
    params = {
        "datasetid": "GHCND",
        "stationid": station_id,
        "startdate": start,
        "enddate": end,
        "limit": 1000,
        "units": "standard",
        "datatypeid": ["TMAX", "TMIN", "PRCP"],
    }
    response = requests.get(base_url, headers=headers, params=params)
    if response.status_code == 200:
        extra_data.extend(response.json().get("results", []))
        print("✅ Success")
    else:
        print(f"❌ Still failed: {response.status_code}")

if extra_data:
    df_retry = pd.DataFrame(extra_data)
    df_retry.to_csv("laguardia_weather_retries.csv", index=False)

## 5. Combining Weather Files

In [ ]:
df_main = pd.read_csv("laguardia_weather_2022.csv")
df_retry = pd.read_csv("laguardia_weather_retries.csv")
df_retry['date'] = pd.to_datetime(df_retry['date'])
weather_retry_wide = df_retry.pivot_table(index='date', columns='datatype', values='value').reset_index()
df_main['date'] = pd.to_datetime(df_main['date'])
full_weather = pd.concat([df_main, weather_retry_wide], ignore_index=True)
full_weather = full_weather.sort_values("date").drop_duplicates("date")
full_weather.to_csv("laguardia_weather_combined_2022.csv", index=False)

## 6. Merging Citi Bike and Weather Data

In [ ]:
bike_df = pd.read_csv("combined_citibike_2022.csv", parse_dates=["started_at"])
weather_df = pd.read_csv("laguardia_weather_combined_2022.csv", parse_dates=["date"])
bike_df['date'] = bike_df['started_at'].dt.date
weather_df['date'] = weather_df['date'].dt.date
merged_df = pd.merge(bike_df, weather_df, on='date', how='left')
merged_df.to_csv("citibike_weather_merged_2022.csv", index=False)
print("✅ Merged dataset shape:", merged_df.shape)

## 7. Observations and Export

- All 36 trip files successfully combined.
- NOAA weather merged on date.
- Ready for dashboard visualization.